<a href="https://colab.research.google.com/github/nyanta012/chatgpt_api_practice/blob/main/section9/section9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colaboratoryのショートカットキー
1. **現在のセルを実行し、次のセルを選択**: `Shift + Enter`
2. **コードセルをテキストセルにする**: `Ctrl + M M`
3. **コードセルを上に追加**: `Ctrl + M A`
4. **コードセルを下に追加**: `Ctrl + M B`
5. **セルの削除**: `Ctrl + M D`

# 演習

FunctionCallingを用いてユーザーからの入力を基に、2つの参照情報を切り替える実装をしてみよう

# API KEYの設定など

In [ ]:
%%capture
!pip install langchain==0.0.340
!pip install llama-index==0.9.7
!wget https://github.com/nyanta012/chatgpt_api_practice/raw/main/pdf/コンプライアンスのすべて.pdf
!wget https://github.com/nyanta012/chatgpt_api_practice/raw/main/pdf/健康のすべて.pdf

In [ ]:
import getpass
import json
import os
from pathlib import Path

from openai import OpenAI
from IPython.display import Markdown, display
from langchain.chat_models import ChatOpenAI
from llama_index import (GPTVectorStoreIndex, LLMPredictor, ServiceContext,
                         StorageContext, VectorStoreIndex, download_loader,
                         load_index_from_storage, PromptTemplate)

apikey = getpass.getpass(prompt="OpenAIのAPIキーを入力してください")
os.environ["OPENAI_API_KEY"] = apikey # llama indexは環境変数じゃないとエラーがでる？

client = OpenAI(api_key=apikey)

OpenAIのAPIキーを入力してください··········


# 実装してみよう

# 実装例(一例)

In [ ]:
# llama indexのpdf readerを使う
PDFReader = download_loader("PDFReader")
loader = PDFReader()

# テキスト情報を抽出する
document_1 = loader.load_data(file=Path("コンプライアンスのすべて.pdf"))
document_2 = loader.load_data(file=Path("健康のすべて.pdf"))

# テキストをインデックス化する
INDEX_1 = VectorStoreIndex.from_documents(document_1)
INDEX_2 = VectorStoreIndex.from_documents(document_2)

In [ ]:
def get_index_engine(index_name: str):
    """
    プロンプトを回答するのに適切なindexを提供する
    """
    TEMPLATE = (
        "下記の情報が与えられています。 \n"
        "---------------------\n"
        "{context_str}"
        "\n---------------------\n"
        "この情報を参照して次の質問に答えてください:  {query_str}\n"
    )
    PROMPT = PromptTemplate(TEMPLATE)

    if index_name == "コンプライアンス":
        return INDEX_1.as_query_engine(text_qa_template=PROMPT)
    elif index_name == "健康問題":
        return INDEX_2.as_query_engine(text_qa_template=PROMPT)
    else:
        return None

In [ ]:
def get_chatgpt_response_from_different_sources(user_input: str) -> None:
    # 与えた関数の説明を記載
    my_functions = [
        {
            "name": "get_index_engine",
            "description": "ユーザーの質問・指示に答えるために使われる関数",
            "parameters": {
                "type": "object",
                "properties": {
                    "index_name": {
                        "type": "string",
                        "enum": ["コンプライアンス", "健康問題", "該当なし"],
                        "description": "ユーザーの質問に答えるために、関連が高い単語を1つ選択してください。",
                    },
                },
                "required": ["index_name"],
            },
        }
    ]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[{"role": "user", "content": user_input}],
        functions=my_functions,
        function_call={"name": "get_index_engine"},
        seed=0
    )

    message = response.choices[0].message
    function_args = json.loads(message.function_call.arguments)

    engine = get_index_engine(function_args["index_name"])

    if engine:
        function_response = engine.query(user_input)
        source = function_response.get_formatted_sources(200)  # 200文字まで表示
        used_index = function_args["index_name"]
        display(
            Markdown(
                f"## ユーザー入力\n\n{user_input}\n\n## AI回答\n\n{function_response}\n\n## 参照情報({used_index}) \n\n{source}"
            )
        )
    else:
        print("関連する情報がありません")

In [ ]:
get_chatgpt_response_from_different_sources("AIとコンプライアンスについて教えてください")

## ユーザー入力

AIとコンプライアンスについて教えてください

## AI回答

AIとコンプライアンスについては、以下のような情報が与えられています。

- 人工知能（AI）の発展により、コンプライアンス活動における自動化や洞察力の向上が期待されています。
- AIによるリスク評価と予測、自動監視とデータ分析、AIの倫理と透明性などがコンプライアンスと関連しています。
- AIの活用に伴う課題やリスク、人間とAIの役割分担に関する考察も取り上げられています。
- AIを活用してコンプライアンス活動を効率化し、洞察力とリスク管理能力を向上させるための手法を学ぶことができます。

具体的な内容や詳細は与えられていませんが、AIがコンプライアンス活動においてどのように活用されるかや、その効果について学ぶことができると言えます。

## 参照情報(コンプライアンス) 

> Source (Doc id: ea8e3d00-0cd5-498f-809f-280d60017a1e): 【35. コンプライアンスとクラウドコンピューティング】 （ 500文字程度）  
クラウドコンピューティングの普及により、データ保護とコンプライアンスの課題が浮上
しています。 この章では、 コンプライアンスとクラウドコンピューティングの関連性につい
て詳しく解説します。 データの保護とプライバシー確保、 クラウドプロバイダとの契約と監
査、データの移転と適法性の確保などに関する具体的なア...

> Source (Doc id: 9c115029-6693-4ef3-9822-516385eb4dc3): テクノロジーの活用とコンプライアンス】 （ 500文字程度）  
テクノロジーの進化は、コンプライアンス活動において革新的な手法とチャンスをもたら
しています。 この章では、 テクノロジーの活用とコンプライアンスの関連性について探求し
ます。 AI（人工知能）や機械学習、ブロックチェーンなどのテクノロジーがコンプライアン
スプロセスやリスク管理にどのように活用されているかを解説します。 また...

In [ ]:
get_chatgpt_response_from_different_sources("AIと健康について教えてください")

## ユーザー入力

AIと健康について教えてください

## AI回答

AI（人工知能）は、健康に関連するさまざまな分野で活用されています。予防医学の一環として、AIは健康リスクの評価や予測、疾病の早期発見、治療計画の立案などに役立てられています。AIは大量のデータを解析し、パターンや傾向を見つけ出すことが得意であり、これによって医療の質を向上させることができます。

また、AIはテレヘルスやモバイルアプリなどの健康ケアのデジタル化にも活用されています。例えば、スマートウォッチやフィットネスアプリは、個人の健康データを収集し、運動量や睡眠の質などをモニタリングすることができます。AIはこれらのデータを解析し、個々の健康状態やリスクを評価し、適切なアドバイスや予防策を提供することができます。

さらに、AIは医療診断や治療の分野でも活用されています。例えば、画像認識技術を用いたAIは、X線やMRIなどの画像を解析し、病変や異常を検出することができます。また、患者の病歴や症状データを解析することで、疾病の診断や治療計画の立案を支援することも可能です。

AIの活用によって、医療の効率性や精度が向上し、個々の健康管理や疾病の予防・治療に役立てることができます。しかし、AIの活用には倫理的な問題やプライバシーの保護などの課題も存在します。適切な規制や倫理的なガイドラインの策定が求められています。

## 参照情報(健康問題) 

> Source (Doc id: fc00fc46-c3ec-4d56-90f1-ffe2cd97c04b): 健康を維持するためには、疾病の予防が重要です。予防医学の手法としては、予防接種やス
クリーニングテストの実施、健康診断の受診などがあります。また、リスクファクターの管
理や健康への適切なアドバイスも予防医学の重要な要素です。自身の健康リスクを正しく
評価し、 予防医学の 原則に基づいた行動を取ることで、 病気や健康問題を予防することが可
能となります。  
 
１４．健康への環境への配慮  ...

> Source (Doc id: 2bf0bd5c-d940-42b0-abb0-0da211dead7e): 健康に関する新しい知識や最新の研究
結果を学び続けることで、自身の健康管理や健康への取り組みを進化させることができま
す。生涯学習の一環として、健康に関するセミナーやワークショップへの参加、専門書やオ
ンラインの健康情報の活用、健康に関連するコミュニ ティや団体への参加などが挙げられ
ます。 自身の健康についての知識を深めることで、 より良い健康状態を維持するための選択
や行動が可能となりま...

In [ ]:
get_chatgpt_response_from_different_sources("にゃんたについて教えてください")

関連する情報がありません
